#### Measuring Expectation values & run a batch of circuits

In [2]:
import numpy as np

from braket.circuits import Circuit, Observable
from braket.quantum_information import PauliString
from braket.parametric import FreeParameter, FreeParameterExpression

In [2]:
# Create a small circuit with a free parameter theta
theta = FreeParameter("theta")
pauli_string = PauliString('XY')
pauli_operator = pauli_string.to_unsigned_observable()

circ = Circuit()
circ.rx(0, theta)
circ.expectation(observable=pauli_operator, target=[0, 1])

# Small value for test purposes
batchsize = 5
batch_circuits = [circ] * batchsize

In [3]:
print(circ)

T  : |    0    |  Result Types  |
                                 
q0 : -Rx(theta)-Expectation(X@Y)-
                |                
q1 : -----------Expectation(X@Y)-

T  : |    0    |  Result Types  |

Unassigned parameters: [theta].


In [4]:
thetas = np.arange(0.0*np.pi, 2.5*np.pi, np.pi/2).tolist()[:batchsize]

from braket.devices import LocalSimulator

device = LocalSimulator()

job = device.run(circ, shots=100, inputs={'theta': thetas[0]})

# Run multiple circuits at once
# job = device.run_batch(
#     batch_circuits,
#     inputs={'theta': thetas}, 
#     shots=100
# )
job.result()

SyntaxError: incomplete input (382985951.py, line 15)

#### Custom Pulse-Calibrations

In [368]:
import os
import sys

from braket.aws import AwsQuantumJob, AwsSession
from braket.jobs.local import LocalQuantumJob
from braket.jobs.image_uris import Framework, retrieve_image
from qiskit_braket_provider.providers import adapter
from braket.circuits import Gate, Instruction, Circuit

aws_session = AwsSession(default_bucket="amazon-braket-us-west-1-lukasvoss")

module_path = os.path.abspath(os.path.join('/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control'))
if module_path not in sys.path:
    sys.path.append(module_path)

from needed_files.quantumenvironment import QuantumEnvironment
from needed_files.helper_functions import load_agent_from_yaml_file
from needed_files.ppo import make_train_ppo
from needed_files.q_env_config import q_env_config as gate_q_env_config
from needed_files.pulse_config import q_env_config as pulse_q_env_config

import time

/Users/lukasvoss/anaconda3/envs/braket/lib/python3.10/site-packages/qiskit_dynamics/dispatch/backends/jax.py:34: UserWarning: The functionality in the perturbation module of Qiskit Dynamics requires a JAX version <= 0.4.6, due to a bug in JAX versions > 0.4.6. For versions 0.4.4, 0.4.5, and 0.4.6, using the perturbation module functionality requires setting os.environ['JAX_JIT_PJIT_API_MERGE'] = '0' before importing JAX or Dynamics.
  warnings.warn(
I0000 00:00:1706692955.311062       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


Starting Rabi experiment for qubit 0...
Rabi experiment for qubit 0 done.
Starting Drag experiment for qubit 0...
Drag experiments done for qubit 0 done.
Starting Rabi experiment for qubit 1...
Rabi experiment for qubit 1 done.
Starting Drag experiment for qubit 1...
Drag experiments done for qubit 1 done.
All single qubit calibrations are done
Updated Instruction Schedule Map <InstructionScheduleMap(1Q instructions:
  q0: {'s', 'measure', 'tdg', 'rz', 'id', 'z', 'sdg', 't', 'h', 'x', 'sx', 'reset', 'delay'}
  q1: {'s', 'measure', 'tdg', 'rz', 'id', 'z', 'sdg', 't', 'h', 'x', 'sx', 'reset', 'delay'}
Multi qubit instructions:
  (0, 1): {'ecr', 'cr45m', 'cr45p'}
  (1, 0): {'ecr', 'cr45m', 'cr45p'}
)>


In [369]:
q_env = QuantumEnvironment(pulse_q_env_config)

SparsePauliOp(['I', 'Z'],
              coeffs=[ 0.5+0.j, -0.5+0.j])


In [ ]:
print('Target Gate:', q_env.target['gate'].name)

In [ ]:
q_env.circuit_truncations[0].draw('mpl')

In [ ]:
q_env.parametrized_circuit_func

In [ ]:
# def apply_parametrized_circuit(
#     qc: QuantumCircuit, params: ParameterVector, tgt_register: QuantumRegister, **kwargs
# ):
#     """
#     Define ansatz circuit to be played on Quantum Computer. Should be parametrized with qiskit ParameterVector
#     This function is used to run the QuantumCircuit instance on a Runtime backend
#     :param qc: Quantum Circuit instance to add the gate on
#     :param params: Parameters of the custom Gate
#     :param tgt_register: Quantum Register formed of target qubits
#     :return:
#     """
#     target = kwargs["target"]
#     backend = kwargs["backend"]

#     gate, physical_qubits = target["gate"], target["register"]
#     parametrized_gate = Gate(
#         f"custom_{gate.name}", len(tgt_register), params=params.params
#     )
#     parametrized_schedule = custom_schedule(
#         backend=backend, physical_qubits=physical_qubits, params=params
#     )
#     qc.add_calibration(parametrized_gate, physical_qubits, parametrized_schedule)
#     qc.append(parametrized_gate, tgt_register)

In [ ]:
from typing import List, Optional, Union, cast
from qiskit import QuantumCircuit

In [ ]:
def apply_parametrized_circuit_aws(
        qc: Union[Circuit, QuantumCircuit], target, backend
):
    if isinstance(qc, QuantumCircuit):
        qc = adapter.convert_qiskit_to_braket_circuit(qc)

    # target = kwargs["target"]
    # backend = kwargs["backend"]

    gate, physical_qubits = target["gate"], target["register"]

    own_gate = Gate(
       qubit_count=len(physical_qubits), ascii_symbols=str.upper(gate.name)
    )

    qc.add_instruction(Instruction(own_gate, physical_qubits))

    print('qc:', qc)

In [ ]:
# Start with a qiskit circuit
qc = QuantumCircuit(1)
qc.h(0)
qc.draw('mpl')

In [ ]:
apply_parametrized_circuit_aws(qc, target=q_env.target, backend=q_env.backend)

In [ ]:
physical_qubits = q_env.target['register']
gate = q_env.target['gate'].name

In [ ]:
own_gate = Gate(
    qubit_count=len(physical_qubits), ascii_symbols=str.upper(gate.name)
)

In [ ]:
own_gate.name

In [ ]:
my_circuit = Circuit().add_instruction(Instruction(own_gate, physical_qubits))
print(my_circuit)

In [236]:
from braket.aws import AwsDevice

from braket.pulse import PulseSequence
from braket.circuits import Circuit, GateCalibrations, QubitSet
from braket.circuits.gates import X, Rx, Rz, CNot, XY, PulseGate, U, RZX

import math

In [6]:
# XXX: OQC does not support native gate calibration !
# device = AwsDevice("arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy")

device = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-3")

In [7]:
calibrations = device.gate_calibrations
print(
    f"The number of pulse implementations in the calibration file is {len(calibrations)}."
)

The number of pulse implementations in the calibration file is 925.


In [8]:
calibrations.pulse_sequences

{(Rx('angle': 1.5707963267948966, 'qubit_count': 1),
  QubitSet([Qubit(0)])): <braket.pulse.pulse_sequence.PulseSequence at 0x13f1e4880>,
 (Rx('angle': -1.5707963267948966, 'qubit_count': 1),
  QubitSet([Qubit(0)])): <braket.pulse.pulse_sequence.PulseSequence at 0x13f1e5180>,
 (Rx('angle': 3.141592653589793, 'qubit_count': 1),
  QubitSet([Qubit(0)])): <braket.pulse.pulse_sequence.PulseSequence at 0x13f1e62c0>,
 (Rx('angle': -3.141592653589793, 'qubit_count': 1),
  QubitSet([Qubit(0)])): <braket.pulse.pulse_sequence.PulseSequence at 0x13f1e7400>,
 (Rz('angle': theta, 'qubit_count': 1),
  QubitSet([Qubit(0)])): <braket.pulse.pulse_sequence.PulseSequence at 0x13f2086d0>,
 (Rx('angle': 1.5707963267948966, 'qubit_count': 1),
  QubitSet([Qubit(100)])): <braket.pulse.pulse_sequence.PulseSequence at 0x13f20a740>,
 (Rx('angle': -1.5707963267948966, 'qubit_count': 1),
  QubitSet([Qubit(100)])): <braket.pulse.pulse_sequence.PulseSequence at 0x13f20b040>,
 (Rx('angle': 3.141592653589793, 'qubit_co

In [9]:
list(calibrations.pulse_sequences.keys())[0]

(Rx('angle': 1.5707963267948966, 'qubit_count': 1), QubitSet([Qubit(0)]))

In [10]:
calibrations.pulse_sequences[list(calibrations.pulse_sequences.keys())[0]]

In [371]:
import numpy as np

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import ParameterVector

from braket.devices import LocalSimulator
from braket.circuits import Circuit, Gate, Instruction, QubitSet, AngledGate
from braket.circuits import noises

## Rebuild the parametrized circuit in AWS Braket

#### Qiskit Version

In [ ]:
action_vector = np.pi * np.array([0.0, 0.0, 0.5, 0.5, -0.5, 0.5, -0.5]) # np.random.uniform(-np.pi, np.pi, 7)

In [381]:
q_reg = QuantumRegister(2)
baseline = np.pi * np.zeros(len(action_vector))
params = ParameterVector("a", 7)
qc = QuantumCircuit(q_reg, name="custom_cx")
# optimal_params = np.pi * np.array([0.0, 0.0, 0.5, 0.5, -0.5, 0.5, -0.5])
optimal_params = np.pi * np.zeros(7)

qc.u(
    baseline[0] + params[0],
    baseline[1] + params[1],
    baseline[2] + params[2],
    q_reg[0],
)
qc.u(
    baseline[3] + params[3],
    baseline[4] + params[4],
    baseline[5] + params[5],
    q_reg[1],
)

qc.rzx(baseline[6] + params[6], q_reg[0], q_reg[1])

qc = adapter.convert_qiskit_to_braket_circuit(qc)

print(qc)

T  : |     0     |   1    |     2     |3|4|   5    |6|7|
                                                        
q0 : -PHASE(a[2])-Ry(a[0])-PHASE(a[1])---C----------C---
                                         |          |   
q1 : -PHASE(a[5])-Ry(a[3])-PHASE(a[4])-H-X-Rz(a[6])-X-H-

T  : |     0     |   1    |     2     |3|4|   5    |6|7|

Unassigned parameters: [a[0], a[1], a[2], a[3], a[4], a[5], a[6]].


#### Braket Version

In [351]:
my_qc = Circuit()

baseline = np.pi * np.zeros(len(action_vector))
params = [FreeParameter(f"a{i}") for i in range(1, len(action_vector)+1)]

u1 = U(
    baseline[0] + params[0], 
    baseline[1] + params[1], 
    baseline[1] + params[2],
)
u2 = U(
    baseline[3] + params[3], 
    baseline[4] + params[4], 
    baseline[5] + params[5],
)
# rzx = RZX(
#     baseline[6] + params[6]
# )

In [352]:
my_qc.add_instruction(Instruction(u1, [0]))
my_qc.add_instruction(Instruction(u2, [1]))
print(my_qc)

T  : |      0      |
                    
q0 : -U(a1, a2, a3)-
                    
q1 : -U(a4, a5, a6)-

T  : |      0      |

Unassigned parameters: [a1, a2, a3, a4, a5, a6].


In [353]:
def rzx(theta, register):
    """
    RZX gate decomposition for angle theta
    XXX: One cannot use theta/2 as parameter for RZ gate, therefore divide the value of theta by 2 before passing it to this function!
    """
    q1, q2 = register
    qc = Circuit()

    # Step 1: Apply Hadamard gate to the target qubit
    qc.h(q2)

    # Step 2: Apply CNOT gate with q1 as control and q2 as target
    qc.cnot(q1, q2)

    # Step 3: Apply RZ gate with angle theta to the target qubit
    # Since Braket's RZ gate uses full angle, there's no need to divide theta by 2
    qc.rz(q2, angle=theta)

    # Step 4: Apply another CNOT gate with q1 as control and q2 as target
    qc.cnot(q1, q2)

    # Step 5: Apply Hadamard gate to the target qubit again
    qc.h(q2)
    
    return qc

Circuit.register_subroutine(rzx)

my_qc.rzx(baseline[6] + params[6], [0, 1])

print(my_qc)

T  : |      0      |1|2|  3   |4|5|
                                   
q0 : -U(a1, a2, a3)---C--------C---
                      |        |   
q1 : -U(a4, a5, a6)-H-X-Rz(a7)-X-H-

T  : |      0      |1|2|  3   |4|5|

Unassigned parameters: [a1, a2, a3, a4, a5, a6, a7].


Sanity Checks if RZX was correclty implemented

In [361]:
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])

new_circ = Circuit()
check1 = np.isclose(new_circ.rzx(0, [0, 1]).to_unitary(), np.eye(4), atol=1e-8)
new_circ = Circuit()
check2 = np.isclose(new_circ.rzx(2*np.pi, [0, 1]).to_unitary(), -np.eye(4), atol=1e-8)
new_circ = Circuit()
check3 = np.isclose(new_circ.rzx(np.pi, [0, 1]).to_unitary(), -1j * np.kron(Z, X), atol=1e-8)

if check1.all() and check2.all() and check3.all():
    print("RZX gate is implemented correctly!")

RZX gate is implemented correctly!


In [362]:
device = LocalSimulator(backend="braket_dm")

In [364]:
param_names = [str(param) for param in params]
bound_parameters = dict(zip(param_names, action_vector))

In [378]:
print('Braket Circuit: \n\n', my_qc)
print('\n\nQiskit to Braket Circuit: \n\n', qc)

Braket Circuit: 

 T  : |      0      |1|2|  3   |4|5|
                                   
q0 : -U(a1, a2, a3)---C--------C---
                      |        |   
q1 : -U(a4, a5, a6)-H-X-Rz(a7)-X-H-

T  : |      0      |1|2|  3   |4|5|

Unassigned parameters: [a1, a2, a3, a4, a5, a6, a7].


Qiskit to Braket Circuit: 

 T  : |     0     |   1    |     2     |3|4|   5    |6|7|
                                                        
q0 : -PHASE(a[2])-Ry(a[0])-PHASE(a[1])---C----------C---
                                         |          |   
q1 : -PHASE(a[5])-Ry(a[3])-PHASE(a[4])-H-X-Rz(a[6])-X-H-

T  : |     0     |   1    |     2     |3|4|   5    |6|7|

Unassigned parameters: [a[0], a[1], a[2], a[3], a[4], a[5], a[6]].


In [380]:
# run the circuit on the local simulator
task = device.run(my_qc, shots = 1000, inputs=bound_parameters)

# visualize the results
result = task.result()
measurement = result.measurement_counts
print('measurement results:', measurement)

measurement results: Counter({'00': 1000})


/Users/lukasvoss/anaconda3/envs/braket/lib/python3.10/site-packages/braket/default_simulator/simulator.py:324: UserWarning: You are running a noise-free circuit on the density matrix simulator. Consider running this circuit on the state vector simulator: LocalSimulator("default") for a better user experience.
  warnings.warn(


#### Build own Pulse Schedules